<a href="https://colab.research.google.com/github/sfwtopoulos/stocks/blob/master/Stocks_seq_1st_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import math
import warnings
import numpy as np
import time

import pandas_datareader as pdr
#%tensorflow_version 2.x
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
import keras


from keras.layers import LSTM
from keras.models import Sequential,Model
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [36]:
print(keras.__version__)
print(tf.__version__)
#print(tensorflow.compat.v2.__version__)

2.3.0
1.14.0


In [0]:
#Data import from git repo
url = 'https://raw.githubusercontent.com/sfwtopoulos/stocks/master/stocks_dataset/combined.csv'
#df1 = pd.read_csv(url, error_bad_lines=False)
dfstocks = pd.read_csv(url, sep=',')

In [38]:
#fix column names
dfstocks=dfstocks.rename({' AAPL':'Company', ' Close/Last':'Close', ' Volume':'Volume', ' Open':'Open', ' High':'High', ' Low':'Low'}, axis=1);
for col in dfstocks.columns: 
    print(col) 

Date
Close
Volume
Open
High
Low
Company


In [0]:
#drop $sign from values
dfstocks.Close=dfstocks['Close'].astype(str)
dfstocks.Close=dfstocks.Close.apply(lambda x: x.replace('$',''))
dfstocks.Open=dfstocks['Open'].astype(str)
dfstocks.Open=dfstocks.Open.apply(lambda x: x.replace('$',''))
dfstocks.High=dfstocks['High'].astype(str)
dfstocks.High=dfstocks.High.apply(lambda x: x.replace('$',''))
dfstocks.Low=dfstocks['Low'].astype(str)
dfstocks.Low=dfstocks.Low.apply(lambda x: x.replace('$',''))

In [40]:
#drop rows containing nan or header from the csv files
dfstocks=dfstocks[~dfstocks.Low.str.contains("nan")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("Low")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("N/A")]
dfstocks=dfstocks[~dfstocks.Volume.str.contains("N/A")]
dfstocks.reset_index(drop=True, inplace=True)
dfstocks.Date.count()

229269

In [41]:
#sort dataframe based on date and Company Name
dfstocks = dfstocks.sort_values(['Date', 'Company'])
dfstocks.head(100)

,Date,Close,Volume,Open,High,Low,Company
1761,01/02/2013,78.4328,139906732,79.1171,79.2857,77.3757,AAPL
4248,01/02/2013,35.12,13767660,34.92,35.4,34.1,ABBV
6025,01/02/2013,32.05,20266410,32.3,32.45,31.64,ABT
8512,01/02/2013,69.06,4039095,67.59,69.06,67.55,ACN
10999,01/02/2013,38.34,6483720,37.92,38.73,37.92,ADBE
...,...,...,...,...,...,...,...
10747,01/02/2014,59.29,2745895,59.06,59.53,58.94,ADBE
13234,01/02/2014,168.05,1268722,167.33,170.71,166.5937,AGN
15721,01/02/2014,50.71,9196092,50.81,51.3,50.47,AIG
18208,01/02/2014,53.55,1909106,54.09,54.45,53.5,ALL


In [42]:
#Converting Datatypes
dfstocks.Date=pd.to_datetime(dfstocks.Date)
dfstocks.Close=pd.to_numeric(dfstocks.Close)
dfstocks.Volume=pd.to_numeric(dfstocks.Volume)
dfstocks.Open=pd.to_numeric(dfstocks.Open)
dfstocks.High=pd.to_numeric(dfstocks.High)
dfstocks.Low=pd.to_numeric(dfstocks.Low)
dfstocks.dtypes

Date       datetime64[ns]
Close             float64
Volume              int64
Open              float64
High              float64
Low               float64
Company            object
dtype: object

In [43]:
#dfstocks[dfstocks['Company'].str.contains("AMZN")]
sub_split=dfstocks[dfstocks['Company'].str.contains("MDLZ")]
sub_split.isnull().values.any()
sub_split.isna().values.any()
sub_split = sub_split.sort_values(['Date'])
sub_split.reset_index(drop=True, inplace=True)
sub_split.shape[0]

2487

In [44]:
sub_split

,Date,Close,Volume,Open,High,Low,Company
0,2010-02-16,19.3633,54149150,19.1695,19.3633,18.9288,MDLZ
1,2010-02-17,19.1360,24833350,19.2898,19.3499,19.0157,MDLZ
2,2010-02-18,19.2564,20606450,19.1494,19.2564,19.0324,MDLZ
3,2010-02-19,19.3299,15497250,19.2229,19.4168,19.1494,MDLZ
4,2010-02-22,19.3499,27292690,19.3098,19.4435,19.1494,MDLZ
...,...,...,...,...,...,...,...
2482,2019-12-24,54.9900,1833763,54.9400,55.1800,54.7400,MDLZ
2483,2019-12-26,54.8500,3565549,54.9700,55.0741,54.7200,MDLZ
2484,2019-12-27,55.2800,4229449,54.9300,55.3300,54.8000,MDLZ
2485,2019-12-30,55.0000,3104187,54.9200,55.0700,54.5651,MDLZ


In [0]:
# prepare training and testing data sets for LSTM based sequence modeling
#def dimitris_function(time_series, scaling=True,split_ratio=0.9):
def dimitris_function(time_series,split_ratio=0.9):

    series1 = pd.Series(time_series[time_series.columns[0]])
    series2 = pd.Series(time_series[time_series.columns[1]])
    series3 = pd.Series(time_series[time_series.columns[2]])
    series4 = pd.Series(time_series[time_series.columns[3]])
    series5 = pd.Series(time_series[time_series.columns[4]])

    values1 = series1.values
    values1 = values1.reshape((len(values1), 1))
    values2 = series2.values
    values2 = values2.reshape((len(values2), 1))
    values3 = series3.values
    values3 = values3.reshape((len(values3), 1))
    values4 = series4.values
    values4 = values4.reshape((len(values4), 1))
    values5 = series5.values
    values5 = values5.reshape((len(values5), 1))

    #scaler2 --> Close Price Scaler
    scaler1 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled1"] = scaler1.fit_transform(values1)
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled2"]= scaler2.fit_transform(values2)
    scaler3 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled3"]= scaler3.fit_transform(values3)
    scaler4 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled4"]= scaler4.fit_transform(values4)
    scaler5 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled5"]= scaler5.fit_transform(values5)


    time_series = time_series[["scaled1","scaled2","scaled3","scaled4","scaled5"]].values

        
    split_ratio = int(len(time_series) * split_ratio)

    train = time_series[0:split_ratio]
    test = time_series[split_ratio:len(time_series)]
    
    # return train,test,scaler1,scaler2
    return train,test,scaler2


In [0]:
'''batch_size = [64, 128, 256]
epochs = [10, 20, 30]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
neurons = [50, 100, 150, 200]
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer, learn_rate=learn_rate, 
                  dropout_rate=dropout_rate, weight_constraint=weight_constraint, momentum=momentum)
'''
p = {
    'first_neuron': [12, 24, 48]}

In [0]:
def get_seq_model(params,input_shape,train_x,train_y):
    # create and fit the LSTM network
    model = Sequential()
    # samples*timesteps*features
    # https://keras.io/getting-started/sequential-model-guide/
    #relu_advanced=keras.activations.relu(x,)
    model.add(LSTM(input_shape=input_shape,
                   units = params['first_neuron'],  
   #                activation='relu',
                   return_sequences=True
    ))

    # readout layer. TimeDistributedDense uses the same weights for all
    # time steps.
    # model.add(TimeDistributed(Dense(1))) #number of inputs in the 1st layer
    model.add(TimeDistributed(Dense(5)))   
    start = time.time()
    
    model.compile(loss="mse", optimizer=RMSprop(lr=0.001))

    print("> Compilation Time : ", time.time() - start)
    print(model.summary())
    out=seq_lstm_model.fit(train_x, train_y,
                #epochs=20, batch_size=1,
                epochs=250, batch_size=1,
                verbose=1)
    return out,model

In [48]:
'''

def get_seq_model_2(input1,input2,input3,input4,input5):


  model1=input(shape=input1)
#layer gia 1o feature
  lstm1=LSTM(40,return_sequences=True)(model1)
  output1=TimeDistributed(Dense(1))(lstm1)
#layer gia 2o feature
  lstm2=LSTM(40,return_sequences=True)(model1)
  output2=TimeDistributed(Dense(1))(lstm2)
#layer gia 3o feature
  lstm3=LSTM(40,return_sequences=True)(model1)
  output3=TimeDistributed(Dense(1))(lstm3)
  #layer gia 4o feature
  lstm4=LSTM(40,return_sequences=True)(model1)
  output4=TimeDistributed(Dense(1))(lstm4)
  #layer gia 5o feature
  lstm5=LSTM(40,return_sequences=True)(model1)
  output5=TimeDistributed(Dense(1))(lstm5)
  '''


'\n\ndef get_seq_model_2(input1,input2,input3,input4,input5):\n\n\n  model1=input(shape=input1)\n#layer gia 1o feature\n  lstm1=LSTM(40,return_sequences=True)(model1)\n  output1=TimeDistributed(Dense(1))(lstm1)\n#layer gia 2o feature\n  lstm2=LSTM(40,return_sequences=True)(model1)\n  output2=TimeDistributed(Dense(1))(lstm2)\n#layer gia 3o feature\n  lstm3=LSTM(40,return_sequences=True)(model1)\n  output3=TimeDistributed(Dense(1))(lstm3)\n  #layer gia 4o feature\n  lstm4=LSTM(40,return_sequences=True)(model1)\n  output4=TimeDistributed(Dense(1))(lstm4)\n  #layer gia 5o feature\n  lstm5=LSTM(40,return_sequences=True)(model1)\n  output5=TimeDistributed(Dense(1))(lstm5)\n  '

In [0]:
# print(type(sub_split.Close))
sub_split_date=sub_split[['Date']]
sub_split=sub_split[['Volume', 'Close', 'Open', 'High', 'Low']]
#sub_split.columns[1]

In [50]:
# split train and test datasets
train,test,scaler_Close = dimitris_function(sub_split,split_ratio=0.7)

#train
print(len(train))
print(train.shape[0])
print(train.shape)
print(test.shape)
print(train)

1740
1740
(1740, 5)
(747, 5)
[[0.88815584 0.02021012 0.01335138 0.01636724 0.01466068]
 [0.39046289 0.01423411 0.01651313 0.01601441 0.01695744]
 [0.31870301 0.01739958 0.01282311 0.01355248 0.01739882]
 ...
 [0.07876951 0.68608191 0.69959394 0.69877535 0.69367717]
 [0.13961111 0.67241044 0.68067072 0.68508332 0.68019791]
 [0.10580105 0.69607261 0.66910654 0.69890701 0.68019791]]


In [51]:
#We use numpy to reshape our time series into 3D tensors.
train = np.reshape(train,(1,train.shape[0],5))
test = np.reshape(test,(1,test.shape[0],5))

train_x = train[:,:-1,:]
train_y = train[:,1:,:]

test_x = test[:,:-1,:]
test_y = test[:,1:,:]

print("Data Split Complete")

print("train_x shape={}".format(train_x.shape))
print("train_y shape={}".format(train_y.shape))
print("test_x shape={}".format(test_x.shape))
print("test_y shape={}".format(test_y.shape))

Data Split Complete
train_x shape=(1, 1739, 5)
train_y shape=(1, 1739, 5)
test_x shape=(1, 746, 5)
test_y shape=(1, 746, 5)


In [0]:
#pip install talos

In [53]:
p['first_neuron']

[12, 24, 48]

In [55]:
# build RNN model
seq_lstm_model=None
try:
  import talos
  p = {'first_neuron': [12, 24, 48]}
  t = talos.Scan(x=train_x,y=train_y,params=p,model=seq_lstm_model,experiment_name='test')
   # seq_lstm_model = get_seq_model(input_shape=(train_x.shape[1],5))
except:
    print("Model Build Failed. Trying Again")
    seq_lstm_model = get_seq_model(input_shape=(train_x.shape[1],5))



  0%|          | 0/3 [00:00<?, ?it/s]

Model Build Failed. Trying Again


TypeError: ignored

In [0]:
train_y

In [0]:
# train the model
seq_lstm_model.fit(train_x, train_y,
                #epochs=20, batch_size=1,
                epochs=250, batch_size=1,
                verbose=1)
import talos
t = talos.Scan(x=x, y=y, params=p)
print("Model Fit Complete")

In [0]:
# train fit performance
trainPredict = seq_lstm_model.predict(train_x)
trainScore = math.sqrt(mean_squared_error(train_y[0], trainPredict[0]))
print('Train Score: %.2f RMSE' % (trainScore))

In [0]:
trainPredict

In [0]:
# Pad input sequence
testPredict = pad_sequences(test_x,maxlen=train_x.shape[1],padding='post',dtype='float64')

In [0]:
# forecast values
testPredict = seq_lstm_model.predict(testPredict)
testPredict

In [0]:
#Drop all other variables except Close Price Output
testPredict=np.delete(testPredict, [2,3,4], 2)
trainPredict=np.delete(trainPredict, [2,3,4], 2)
# testPredict=np.delete(testPredict, 3, 2)
# trainPredict=np.delete(trainPredict, 3, 2)
# testPredict=np.delete(testPredict, 4, 2)
# trainPredict=np.delete(trainPredict, 4, 2)
testPredict=np.delete(testPredict, 0, 2)
trainPredict=np.delete(trainPredict, 0, 2)

In [0]:
testPredict

In [0]:
trainPredict


In [0]:
type(testPredict)

In [0]:
# # inverse transformation of the Close Price
trainPredict = scaler_Close.inverse_transform(trainPredict.\
                                        reshape(-1,1))
testPredict = scaler_Close.inverse_transform(testPredict.\
                                       reshape(-1,1))

In [0]:
trainPredict

In [0]:
# plot the true and forecasted values
split_ratio = len(trainPredict)+1

plt.plot(sub_split_date.Date,
          sub_split.Close.values,c='black',
          alpha=0.3,label='True Data')

plt.plot(sub_split_date.Date[1:split_ratio],
          trainPredict,label='Training Fit',c='g')

plt.plot(sub_split_date.Date[split_ratio+1:],
          testPredict[:test_x.shape[1]],label='Testing Forecast')
plt.title('Forecast Plot')
plt.legend()
#plt.show()
plt.rcParams["figure.figsize"] = (20,10)
plt.show()

In [0]:
from numpy import loadtxt

dataset = loadtxt("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv", delimiter=",")
x = dataset[:,0:8]
y = dataset[:, 8]

In [0]:
from keras.models import Sequential
from keras.layers import Dense

def diabetes():
    
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, Y, epochs=100, batch_size=10, verbose=0)
    
    return model

In [0]:
from keras.activations import relu, elu

p = {
    'first_neuron': [12, 24, 48],
    'activation': ['relu', 'elu'],
    'batch_size': [10, 20, 30]
}

In [0]:
# add input parameters to the function
def diabetes(x_train, y_train, x_val, y_val, params):
    
    # replace the hyperparameter inputs with references to params dictionary 
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=8, activation=params['activation']))
    #model.add(Dense(8, activation=params['activation']))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # make sure history object is returned by model.fit()
    out = model.fit(x=x, 
                    y=y,
                    validation_data=[x_val, y_val],
                    epochs=100,
                    batch_size=params['batch_size'],
                    verbose=0)
    
    # modify the output model
    return out, model

In [62]:
import talos
t = talos.Scan(x=x, y=y, params=p, model=diabetes, experiment_name='diabetes')




  0%|          | 0/18 [00:00<?, ?it/s]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






  6%|▌         | 1/18 [00:10<02:50, 10.02s/it]


 11%|█         | 2/18 [00:19<02:40, 10.00s/it]


 17%|█▋        | 3/18 [00:30<02:30, 10.02s/it]


 22%|██▏       | 4/18 [00:35<02:01,  8.68s/it]


 28%|██▊       | 5/18 [00:41<01:40,  7.73s/it]


 33%|███▎      | 6/18 [00:46<01:24,  7.08s/it]


 39%|███▉      | 7/18 [00:50<01:07,  6.14s/it]


 44%|████▍     | 8/18 [00:54<00:55,  5.55s/it]


 50%|█████     | 9/18 [00:58<00:46,  5.11s/it]


 56%|█████▌    | 10/18 [01:08<00:52,  6.59s/it]


 61%|██████    | 11/18 [01:19<00:53,  7.64s/it]


 67%|██████▋   | 12/18 [01:29<00:50,  8.41s/it]


 72%|███████▏  | 13/18 [01:34<00:37,  7.59s/it]


 78%|███████▊  | 14/18 [01:40<00:27,  7.00s/it]


 83%|████████▎ | 15/18 [01:46<00:19,  6.60s/it]


 89%|████████▉ | 16/18 [01:50<00:11,  5.82s/it]


 94%|█████████▍| 17/18 [01:54<00:05,  5.27s/it]


100%|██████████| 18/18 [01:58<00:00,  4.92s/it]




In [63]:
t